# Normalize Raw Levelogger and Barologger Data

This notebook ingests raw levelogger and barologger data from each year, outputing new level/baro files with consistent naming conventions, headers, and units. Additionally, if the input data is from a levelogger, a log file will be created and populated with the metadata extracted from the raw input file. **This should be run once at the beginning of the project**

Essentially, this notebook adjusts the raw data to conform to conventions specified in config.py. 

**Note:**
* All processing templates expect data to be formatted according to the conventions implemented by this notebook
* This notebook assumes each year of raw data has its own directory


Author of Template and Underlying Code: Joe Ammatelli | (jamma@uw.edu) | July 2022

## Import Relevant Libraries
**Analyst TODO**: Nothing

In [3]:
import pandas as pd
import os
import sys
import shutil
from datetime import datetime

sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

import config
import level_baro_utils
import log_utils

sys.path.remove(os.path.abspath(os.path.join('..', 'src')))

## Specify path to each year of data
**Analyst TODO**: Create a string mapping between each year ('YYYY') and the corresponding path to its raw data ('path')

In [4]:
# TODO: populate dictionary with mappings between year of data and path to raw data
# Example
# raw_data_dir_by_year = {'2019':'~/tum_raw/raw_19',
#                         '2020':'~/tum_raw/raw_20',
#                         '2021':'~/tum_raw/raw_21'}

raw_data_dir_by_year = {'year' : 'path'}

## Normalize Each Year of Data

**Analyst TODO**: Nothing

In [5]:
for year in raw_data_dir_by_year:
    print('*******************************')
    print(f'Normalizing Data for Year: {year}')
    print('*******************************\n')
    
    # read data from correct year
    data_dir = raw_data_dir_by_year[year]
    
    # write output to correct place
    loc_year_dir_path = os.path.join('..', f'unvented_{year}')
    
    # create directory to store raw data
    raw_dir = os.path.join(loc_year_dir_path, 'data', 'raw')
    if not os.path.exists(raw_dir):
        os.mkdir(raw_dir)
    
    # create dir to store normalized data
    norm_dir = os.path.join(loc_year_dir_path, 'data', 'normalized_raw')
    if not os.path.exists(norm_dir):
        os.mkdir(norm_dir)
    
    # normalize each file in directory
    for fn in os.listdir(data_dir):
        if fn[0] == '.':
            continue
            
        print(f'Normalizing {fn}')
        
        f_path = os.path.join(data_dir, fn)
        
        # infer site
        sitekey = level_baro_utils.infer_site_from_name(fn)
        sitename = None
        if sitekey is None:
            sitename = fn.split('_')[0]
        else:
            sitename = config.SITE_SHORTNAME[sitekey]
        
        # infer sensor type
        sensor_type = level_baro_utils.infer_sensor_type_from_name(fn)
        
        # find header start and read file metadata
        meta, header_num = level_baro_utils.read_meta(f_path)
        
        # read data, convert to desired units, normalize header name
        df = level_baro_utils.read_normalize_solinst_data(f_path, header_num, sensor_type)
    
        # save raw data to
        raw_path = os.path.join(raw_dir, fn)
        shutil.copyfile(f_path, raw_path)
        
        # save normalized file according to naming_convention
        data_name = config.NORMALIZED_FN.format(sensor_type=sensor_type, 
                                                sitename=sitename, 
                                                year=year)
        data_path = os.path.join(norm_dir, data_name)
        df.to_csv(data_path)
        
        # create log file for site and append meta data
        if sensor_type == 'lvl':
            log_dir = os.path.join(loc_year_dir_path, 'logs')
            log_name = f'{sitename}_{year}_log.txt'
            log_path = os.path.join(log_dir, log_name)
            log_utils.create_site_log(log_path, sitename, year)
            log_utils.append_to_log(log_path, 'RAW SOLINST META DATA\n', ''.join(meta))
        
        print(f'Finished Normalizing {fn}\n')

*******************************
Normalizing Data for Year: 2019
*******************************



FileNotFoundError: [Errno 2] No such file or directory: '../unvented_2019/data/raw'